In [1]:
import sys
sys.path.insert(0, '/home/jdawson/')
from py.s3_helper import files_list, print_files_list, read_file

In [2]:
import pandas as pd
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [3]:
BASE = 'unitioplatform-emr-raw/data/utah/20250313/'

# Load the data using your method
df_pat = pd.read_csv(read_file(BASE + 'zisr4x_patients_20250312_v_0_3_0.txt', Bucket='t1dudp-dev-backup'),
                     sep='|', dtype=str)

In [4]:
# Define the base path
BASE = 'unitioplatform-emr-raw/data/utah/20250313/'

# Load the data using your method
df_enc = pd.read_csv(read_file(BASE + 'zisr4x_encounters_20250312_v_0_3_0.txt', Bucket='t1dudp-dev-backup'),
                     sep='|', dtype=str)

In [5]:
BASE = 'unitioplatform-emr-raw/data/utah/20250313/'

# Load the data using your method
df_obs = pd.read_csv(read_file(BASE + 'zisr4x_observations_20250312_v_0_3_0.txt', Bucket='t1dudp-dev-backup'),
                     sep='|', dtype=str)

In [6]:
# Define the base path
BASE = 'unitioplatform-emr-raw/data/utah/20250313/'

# Load the data using your method
df_dia = pd.read_csv(read_file(BASE + 'zisr4x_diabetes_20250312_v_0_3_0.txt', Bucket='t1dudp-dev-backup'),
                     sep='|', dtype=str)


In [7]:
# Define the base path
BASE = 'unitioplatform-emr-raw/data/utah/20250313/'

# Load the data using your method
df_med = pd.read_csv(read_file(BASE + 'zisr4x_medications_20250312_v_0_3_0.txt', Bucket='t1dudp-dev-backup'),
                     sep='|', dtype=str)

In [9]:
import pandas as pd

# ------------------------
# 1. Setup and Filtering
# ------------------------

df_enc["encounter_date"] = pd.to_datetime(df_enc["encounter_date"], format="%Y%m%d", errors="coerce")
df_enc["endo_visit"] = pd.to_numeric(df_enc["endo_visit"], errors="coerce")

mask = (
    (df_enc["encounter_date"].dt.year == 2023) &
    (df_enc["status"] == "finished") &
    (df_enc["endo_visit"] != 0)
)

recent_encs = df_enc.loc[mask].copy()
recent_encs.sort_values(by=["patient_id", "encounter_date"], ascending=[True, False], inplace=True)
recent_encs = recent_encs.drop_duplicates(subset="patient_id", keep="first")
denominator = len(recent_encs)

recent_encs = recent_encs.merge(df_pat[["patient_id", "birth_date"]], on="patient_id", how="left")
recent_encs["birth_date"] = pd.to_datetime(recent_encs["birth_date"], errors="coerce")
recent_encs["age"] = ((recent_encs["encounter_date"] - recent_encs["birth_date"]).dt.days // 365)

recent_encs_depression = recent_encs[(recent_encs["age"] >= 13) & (recent_encs["age"] <= 85)].copy()
denom_depression = len(recent_encs_depression)
pats_depression = recent_encs_depression["patient_id"]

# ------------------------
# 2. Helper Functions
# ------------------------

def coverage(df, col, pats, condition=None):
    sub = df[df["patient_id"].isin(pats)].copy()
    try:
        sub[col] = pd.to_numeric(sub[col])
    except Exception:
        pass
    if condition is not None:
        sub = sub[condition(sub[col])]
    return sub["patient_id"].nunique()

def coverage_within_days(df, code, recent_encs_df, days=365):
    df = df.copy()
    df["effective_date"] = pd.to_datetime(df["effective_date"], errors="coerce")
    df_code = df[df["code"] == code].copy()
    merged = df_code.merge(recent_encs_df[["patient_id", "encounter_date"]], on="patient_id")
    merged["days_diff"] = (merged["encounter_date"] - merged["effective_date"]).dt.days
    recent_hits = merged[(merged["days_diff"] >= 0) & (merged["days_diff"] <= days)]
    return recent_hits["patient_id"].nunique()

def coverage_multi(df, codes, recent_encs_df, days=365):
    df = df.copy()
    df["effective_date"] = pd.to_datetime(df["effective_date"], errors="coerce")
    df_code = df[df["code"].isin(codes)].copy()
    merged = df_code.merge(recent_encs_df[["patient_id", "encounter_date"]], on="patient_id")
    merged["days_diff"] = (merged["encounter_date"] - merged["effective_date"]).dt.days
    recent_hits = merged[(merged["days_diff"] >= 0) & (merged["days_diff"] <= days)]
    return recent_hits["patient_id"].drop_duplicates().nunique()


def coverage_latest(df, code, recent_encs_df, condition=None, days=365):
    df = df.copy()
    df["effective_date"] = pd.to_datetime(df["effective_date"], errors="coerce")
    recent_encs_df["encounter_date"] = pd.to_datetime(recent_encs_df["encounter_date"], errors="coerce")

    # Filter to code and merge with encounters
    df_code = df[df["code"] == code]
    merged = df_code.merge(recent_encs_df[["patient_id", "encounter_date"]], on="patient_id")
    merged["days_diff"] = (merged["encounter_date"] - merged["effective_date"]).dt.days
    merged = merged[(merged["days_diff"] >= 0) & (merged["days_diff"] <= days)]

    # Sort and keep latest per patient
    merged.sort_values(["patient_id", "effective_date"], inplace=True)
    latest = merged.groupby("patient_id").tail(1)
    if condition is not None:
        try:
            latest["value"] = pd.to_numeric(latest["value"])
        except Exception:
            pass
        latest = latest[condition(latest["value"])]

    return latest["patient_id"].nunique()

# ------------------------
# 3. Metrics
# ------------------------

results = {}
pats = recent_encs["patient_id"]

# Demographics
results["birth_date"] = coverage(df_pat, "birth_date", pats)
results["race_1"] = coverage(df_pat, "race_1", pats)
results["race_2"] = coverage(df_pat, "race_2", pats)
results["ethnicity"] = coverage(df_pat, "ethnicity", pats)
results["primary_insurance_type"] = coverage(df_pat, "primary_insurance_type", pats)
results["t1d_dx_dt"] = coverage(df_dia, "t1d_dx_dt", pats)

# BMI
results["BMI"] = coverage_within_days(df_obs, "39156-5", recent_encs)

# A1c
results["Lab A1c"] = coverage_multi(df_obs, ["4548-4", "17856-6"], recent_encs)
results["POC A1c"] = coverage_within_days(df_obs, "poca1c", recent_encs)

# CGM
results["cgm_binary=1 (latest)"] = coverage_latest(df_dia, "cgm_binary", recent_encs, condition=lambda s: s == 1)
results["cgm_st_dt"] = coverage_within_days(df_dia, "cgm_st_dt", recent_encs)
results["cgm_company"] = coverage_within_days(df_dia, "cgm_company", recent_encs)
results["cgm_model"] = coverage_within_days(df_dia, "cgm_model", recent_encs)

# BG Monitoring
results["bgm_test_freq"] = coverage_within_days(df_dia, "bgm_test_freq", recent_encs)

# Pump/MDI
results["ins_regimen=1 (latest)"] = coverage_latest(df_dia, "ins_regimen", recent_encs, condition=lambda s: s == 1)
results["pump_st_dt"] = coverage_within_days(df_dia, "pump_st_dt", recent_encs)
results["pump_company"] = coverage_within_days(df_dia, "ins_pump_company", recent_encs)
results["pump_model"] = coverage_within_days(df_dia, "ins_pump_model", recent_encs)
results["HCL use"] = coverage(df_dia[df_dia["code"] == "ins_pump_delivery"], "value", pats, lambda s: s == 4)
results["ins_regimen!=1"] = coverage(df_dia[df_dia["code"] == "ins_regimen"], "value", pats, lambda s: s != 1)

# Depression Screening
results["Depression screening"] = coverage_multi(df_obs, ["55758-7", "44261-6"], recent_encs_depression)

# Phase 2
results["time_in_range"] = coverage_within_days(df_dia, "time_in_range", recent_encs)
results["cgm_below_70"] = coverage_within_days(df_dia, "cgm_below_70", recent_encs)
results["cgm_below_54"] = coverage_within_days(df_dia, "cgm_below_54", recent_encs)
results["dka_events_inp"] = coverage(df_dia[df_dia["code"] == "dka_events_inp"], "value", pats, lambda s: s != 0)
results["dka_events_amb"] = coverage(df_dia[df_dia["code"] == "dka_events_amb"], "value", pats, lambda s: s != 0)
results["dka_events_inp_pro"] = coverage(df_dia[df_dia["code"] == "dka_events_inp_pro"], "value", pats, lambda s: s != 0)
results["dka_events_amb_pro"] = coverage(df_dia[df_dia["code"] == "dka_events_amb_pro"], "value", pats, lambda s: s != 0)
results["bolus_ins_daily_inj"] = coverage_within_days(df_dia, "bolus_ins_daily_inj", recent_encs)

# SDOH
results["SDOH"] = coverage_multi(df_obs, ["88124-3", "88122-7", "88123-5"], recent_encs)

# Medications
results["drug_name"] = coverage(df_med, "drug_name", pats)
results["drug_name_generic"] = coverage(df_med, "drug_name_generic", pats)
results["drug_class"] = coverage(df_med, "drug_class", pats)
results["drug_sub_class"] = coverage(df_med, "drug_sub_class", pats)

# ------------------------
# 4. Output Prep
# ------------------------

report = pd.DataFrame.from_dict(results, orient="index", columns=["Numerator"])
report["Denominator"] = denominator
report.loc["Depression screening", "Denominator"] = denom_depression
report["Percent"] = round((report["Numerator"] / report["Denominator"]) * 100, 1)

# Add Code and Group columns for export
report = report.reset_index()
report.rename(columns={"index": "Code"}, inplace=True)

def assign_group(code):
    if code in ["birth_date", "race_1", "race_2", "ethnicity", "primary_insurance_type", "t1d_dx_dt"]:
        return "Demographic data"
    elif code == "BMI":
        return "BMI data"
    elif "A1c" in code:
        return "A1c data"
    elif "cgm" in code or "time_in_range" in code:
        return "CGM data"
    elif "bgm" in code:
        return "BG Monitoring"
    elif "pump" in code or "ins_regimen" in code or "HCL" in code or "MDI" in code:
        return

/tmp/ipykernel_9181/3503951261.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  latest["value"] = pd.to_numeric(latest["value"])
/tmp/ipykernel_9181/3503951261.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  latest["value"] = pd.to_numeric(latest["value"])


In [10]:
from docx import Document

class Metric:
    def __init__(self, name, numerator, denominator, code=None):
        self.name = name
        self.numerator = numerator
        self.denominator = denominator
        self.percent = int(round((numerator / denominator) * 100)) if denominator else "-"
        self.code = code


class MetricReport:
    def __init__(self, title):
        self.title = title
        self.metrics = []

    def add_metric(self, metric):
        self.metrics.append(metric)

    def from_dataframe(self, df):
        for _, row in df.iterrows():
            self.add_metric(Metric(
                name=row["Code"],
                numerator=row["Numerator"],
                denominator=row["Denominator"],
                code=row["Code"]
            ))

    def export_to_word(self, filename):

        # --- YOUR GROUPING RULES (MERGE BASED ON THESE) ---
        groups = {
            "Patient Data": [
                "birth_date", "race_1", "race_2", "ethnicity",
                "primary_insurance_type", "t1d_dx_dt", "BMI"
            ],
            "HbA1c": ["Lab A1c", "POC A1c"],
            "Device Data – CGM": [
                "cgm_binary=1", "cgm_st_dt", "cgm_company", "cgm_model",
                "time_in_range", "cgm_below_70", "cgm_below_54"
            ],
            "Device Data – BGM": ["bgm_test_freq"],
            "Device Data – Pump": [
                "ins_regimen=1", "pump_st_dt", "pump_company",
                "pump_model", "HCL use"
            ],
            "Insulin Injection Data": ["ins_regimen!=1", "bolus_ins_daily_inj"],
            "DKA Events": [
                "dka_events_inp", "dka_events_amb",
                "dka_events_inp_pro", "dka_events_amb_pro"
            ],
            "Medication Data": [
                "drug_name", "drug_name_generic", "drug_class", "drug_sub_class"
            ],
            "Social Needs (SDOH)": ["SDOH"],
            "Depression": ["Depression screening"]
        }

        # --- Create the doc ---
        doc = Document()
        doc.add_heading(self.title, level=1)
        doc.add_paragraph(
            "This document shows the unique patient coverage over a calendar year. "
            "The denominator is the number of unique patients that had a finished outpatient, "
            "virtual, or education visit in the reporting year."
        )

        # --- Summary table ---
        summary = doc.add_table(rows=1, cols=2)
        summary.style = "Table Grid"
        summary.cell(0, 0).text = "Annual total unique patient endo encounters"
        summary.cell(0, 1).text = str(int(self.metrics[0].denominator))

        # --- Main detailed table ---
        table = doc.add_table(rows=len(self.metrics) + 1, cols=4)
        table.style = "Table Grid"

        # Header
        hdr = table.rows[0].cells
        hdr[0].text = "Measure"
        hdr[1].text = "Codes"
        hdr[2].text = "Data availability"
        hdr[3].text = "Meets Mapping Expectations"

        # Populate rows
        for i, m in enumerate(self.metrics, start=1):
            row = table.rows[i].cells
            row[1].text = m.name
            row[2].text = f"{m.percent}%" if isinstance(m.percent, int) else "-"
            row[3].text = (
                f"{int(m.numerator)}/{int(m.denominator)}"
                if m.denominator else "Not Available"
            )

        # --- Apply merging based on your grouping dictionary ---
        for group_name, code_list in groups.items():

            matching_rows = []
            for i in range(1, len(table.rows)):     # skip header
                code_value = table.cell(i, 1).text.strip()
                if code_value in code_list:
                    matching_rows.append(i)

            if matching_rows:
                top_cell = table.cell(matching_rows[0], 0)
                top_cell.text = group_name
                for r in matching_rows[1:]:
                    top_cell.merge(table.cell(r, 0))

        doc.save(filename)


In [11]:
# Create report object
word_report = MetricReport("T1Dx Data Availability Report – 2024")

# Populate from your final DataFrame
word_report.from_dataframe(report)

# Export to Word
word_report.export_to_word("T1Dx_Data_completeness_scorecard_2024.docx")